# Importing the required packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# plate size, m
w = h = 0.010
# intervals in x-, y- directions, mm
dx = dy = 0.0002
# Thermal diffusivity of steel, mm2.s-1
K = 4.*0.001*0.001
rho=1.0
cp=1.0
Tcool, Thot = 300, 700
nsteps = 20000

In [3]:
nx, ny = int(w/dx), int(h/dy)
dx2, dy2 = dx*dx, dy*dy
dt = dx2 * dy2 / (2 * K * (dx2 + dy2)/rho/cp)
T0 = Tcool * np.ones((nx, ny))
T = np.empty((nx, ny))
q = np.zeros((nx, ny))

In [4]:
# Initial conditions - ring of inner radius r, width dr centred at (cx,cy) (mm)
r, cx, cy = 0.002, 0.005, 0.005
r2 = r**2
for i in range(nx):
    for j in range(ny):
        p2 = (i*dx-cx)**2 + (j*dy-cy)**2
        if p2 < r2:
            T0[i,j] = Thot

In [5]:
def do_timestep(T0, T):
    # Propagate with forward-difference in time, central-difference in space
    T[1:-1, 1:-1] = T0[1:-1, 1:-1] + (K * dt/rho/cp) * (
          (T0[2:, 1:-1] - 2*T0[1:-1, 1:-1] + T0[:-2, 1:-1])/dx2
          + (T0[1:-1, 2:] - 2*T0[1:-1, 1:-1] + T0[1:-1, :-2])/dy2 ) +q[1:-1,1:-1]*(dt/rho/cp)

    T0 = T.copy()
    return T0, T

# Solve the equations to generate the temperature field

In [6]:
X=np.zeros((nsteps,nx,ny))
for m in range(nsteps):
    T0, T = do_timestep(T0, T)
    X[m,:,:]=T[:,:]

# Preparing data for training

In [7]:
X_train=np.reshape(X, (len(X), nx, ny, 1))

In [8]:
# We want to predict the temperature field at time "n+1" using the temperature field at time "n"
X1_train=X_train[0:len(X_train)-1,:,:,:]
y1_train=X_train[1:len(X_train),:,:,:]

In [9]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
input_img = Input(shape=(nx, ny, 1))
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu',padding='same')(x)
decoded = Conv2D(1, (3, 3), activation='linear', padding='same')(x)
autoencoder = Model(input_img, decoded)
autoencoder.compile(loss='mean_squared_error', optimizer='adam')

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
autoencoder.fit(X1_train, y1_train,epochs=10,batch_size=500)

Epoch 1/10


In [ ]:
plt.contourf(X1_train[0,:,:,0])
plt.show()

# Make the predictions using CNN

In [ ]:
xtest=X1_train[0,:,:,0]
xtest=np.reshape(xtest,(1,nx,ny,1))
Xconv=np.zeros((nsteps,nx,ny))
for m in range(nsteps):
    xtest=autoencoder.predict(xtest)
    Xconv[m,:,:]=xtest[0,:,:,0]

In [ ]:
plt.contourf(Xconv[200,:,:])
plt.show()

In [ ]:
plt.plot(Xconv[0:2000,25,25])
plt.plot(X1_train[0:2000,25,25,0])
plt.show()